In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr, kendalltau


In [6]:
# load images

lables_path = 'data/LDCTIQAG2023_train/train.json'
images_dir = 'data/LDCTIQAG2023_train/image'

def load_images(data):
    
    image_np = list()
    for name,val in data.items():
        img = cv2.imread(f'{images_dir}/{name}', cv2.IMREAD_UNCHANGED)
        image_np.append(img)
    return np.stack(image_np)

with open(lables_path) as file:
    all_lables = pd.Series(json.load(file))

In [41]:
sz = all_lables.shape[0]
dist_array = np.empty((sz*(sz-1))//2)
with tqdm(enumerate(all_lables.items()), desc='Calculating distances') as loops:
    processed = list()
    for key, item in loops:
        name, el = item
        loops.set_description(f"Epoch {el}")
        for rel_key, relative in enumerate(processed):
            position = sz*rel_key+key-((rel_key+2)*(rel_key+1))//2
            distance = el-relative
            dist_array[position] = 0 if distance==0 else distance/abs(distance)
        
        processed.append(el)
dist_array

Epoch 1.8: : 1000it [00:03, 272.83it/s]  


array([-1.,  1., -1., ..., -1., -1.,  1.])

In [113]:
indexes = np.arange(sz)
for j in range(1,sz):
    for i in range(j):
        el_i = indexes[i]
        el_j = indexes[j]
        dist_idx = sz*el_i+el_j-((el_i+2)*(el_i+1))//2
        if dist_array[dist_idx] == -1:
            indexes[i],indexes[j] = indexes[j],indexes[i]


In [112]:
el_i = indexes[21]
el_j = indexes[188]
dist_idx = sz*el_i+el_j-((el_i+2)*(el_i+1))//2
dist_array[dist_idx]

-1.0

In [133]:
len(indexes)

1000

In [134]:
sz

1000

In [139]:
def validate_vector(total_pred, total_gt):
    aggregate_results=dict()
    aggregate_results["plcc"] = abs(pearsonr(total_pred, total_gt)[0])
    aggregate_results["srocc"] = abs(spearmanr(total_pred, total_gt)[0])
    aggregate_results["krocc"] = abs(kendalltau(total_pred, total_gt)[0])
    aggregate_results["overall"] = abs(pearsonr(total_pred, total_gt)[0]) + abs(spearmanr(total_pred, total_gt)[0]) + abs(kendalltau(total_pred, total_gt)[0])
    return aggregate_results

In [140]:
sz = all_lables.shape[0]
result = np.zeros(sz, dtype=np.int64)
insert = 0
result[indexes[0]] = 0
for i in range(1, len(indexes)):
    
    el_i = indexes[i-1]
    el_j = indexes[i]
    dist_idx = sz*el_i+el_j-((el_i+2)*(el_i+1))//2
    insert+=dist_array[dist_idx]
    result[indexes[i]] = insert
print(result[:10])

[165  90 202 110 110   1  10 226   0  79]


In [143]:
validate_vector(np.array(result), all_lables.values)

{'plcc': 0.9944939571607939,
 'srocc': 0.9985294294286472,
 'krocc': 0.9765676640331395,
 'overall': 2.9695910506225807}

In [130]:
dist_array

array([-1.,  1., -1., ..., -1., -1.,  1.])

In [131]:
result

array([140158717088704, 140158717088704,  94718119431024,  94718119431024,
                     1,  94718119431088,  94718119431536,               1,
                   237,  94718119431120,  94716913778712,  94718072954464,
       140157667770370,               0,               2,     25769803778,
       140157667770392, 140158683521968,               1,      8589934593,
           38654705666,     47244640258, 140157667770389,  94718119431168,
        94718072665440,      8589934593,     38654705666,     73014444034,
                     1,  94718119431280,  94718119431288,      8589934616,
       140158708983856,     98784247809, 140157667770390,     77309411330,
           85899345922,               1,  94718119431352,  94718119431456,
       140157667770392, 140158683521968,  94716913778689,               0,
          120259084290,    128849018882,  94716913778709,  94718119431360,
       140158706928816,               1,    120259084290,    154618822658,
       140158683521648,  

In [114]:
indexes[:10]

array([  8,  75, 187, 188, 208, 266, 281, 294, 314, 370])

In [60]:
i=0
indexes[i],indexes[i+1] = indexes[i+1],indexes[i] 

In [80]:
all_lables.values

array([2.8, 1.8, 3.4, 2. , 2. , 0.2, 0.6, 4. , 0. , 1.6, 4. , 2.2, 0.6,
       2.4, 1.2, 4. , 2. , 0.6, 1.4, 3.6, 1.6, 0.2, 3.8, 2.2, 1. , 3.8,
       1.4, 2.2, 2.2, 0.4, 3.2, 1.8, 1.8, 0.8, 1.6, 1.4, 1. , 4. , 1. ,
       1.8, 1.6, 2.8, 3.2, 3.4, 1.8, 1.4, 0.6, 1.4, 2.4, 2.8, 1.8, 2.6,
       2.4, 3.2, 1.6, 3.2, 2.2, 1.4, 1.4, 3.4, 2. , 0.6, 1.2, 1.8, 2.6,
       3. , 2. , 0.8, 2.4, 2.6, 3.8, 1.4, 2.6, 0.4, 1.4, 0. , 2.6, 2.4,
       3. , 1.2, 0.4, 0.2, 3.6, 1.4, 2.4, 3.6, 3. , 3. , 2.8, 3. , 1.2,
       3.6, 1.4, 2.6, 0.2, 0.8, 2. , 4. , 1.8, 3. , 4. , 4. , 3.6, 3. ,
       1.8, 1.4, 3. , 0.8, 4. , 2. , 3. , 0.2, 1.2, 1.8, 3. , 0.8, 2.4,
       1.2, 3. , 0.2, 2.2, 3.6, 4. , 2.6, 1.4, 2.4, 1.6, 0.4, 3.8, 1. ,
       2. , 3.4, 2.2, 2.8, 2. , 1. , 3. , 0.8, 2. , 1. , 2.4, 1.8, 1.2,
       2.6, 1.2, 1.8, 3.4, 1.2, 0.4, 2.6, 0.4, 2.6, 2.6, 2. , 3. , 2. ,
       2.6, 2.6, 3.6, 2.6, 0.6, 2.4, 3.2, 3.8, 2.4, 1.4, 4. , 3. , 1.6,
       2.4, 1.6, 3.8, 2.6, 2.6, 1. , 2.2, 2.4, 3.8, 1.4, 1. , 4.

In [84]:
indexes

array([  5,   5,   5,   5,   5,   5,   5,   8,   8,   8,   8,   5,   5,
         5,   5,   8,   8,   6,   8,   8,   8,   8,   8,   6,   8,   8,
         8,   8,   8,   6,   6,   6,   8,   6,   8,   6,   6,   8,   6,
         6,   6,   6,   6,   8,   8,   6,   6,   8,   8,   8,   8,  29,
         6,  29,   6, 343,   6,   6,   6,   6,   6,   6,   6,   6,  29,
         6,   6,   6,  29,  29,  29,  29,  29,  29,  29,  29,   6,  29,
        29,  29,  29,  29,  24,  24,  24, 343,  24,  24,  14,  24,  24,
        24,  24,  14,  14,  24,  24,  24,  24,  24,  24,  24,  24,  24,
        24,  24,  24,  24,  24,  24, 343, 332,  24,  24,  24,  24,  24,
        24,  24,  24,  14,  33, 343,  33,  14,  14,  14,  24,  14,  14,
        24,  24,  24,  24,  24, 343, 332,  24, 332,  24,  24,  24,  24,
        24,  24, 343,  24,  33,   1,   1,  24,  24,  14,   1,  14,  14,
        14,  14,  14,  14,  14,  24,  33,  24,  14, 343,   1,  14,  14,
        14, 343, 332,  33,  14, 332,  24,  24,  14,  33,  14,  1

In [58]:
(dist_array==1).sum()

233794

In [44]:
(dist_array==-1).sum()

239993

In [39]:
distance

NameError: name 'distance' is not defined

In [40]:
(el-relative)/abs(el-relative)

1.0

In [36]:
2//0

ZeroDivisionError: integer division or modulo by zero

In [34]:
dist_array.sum()

499500.0

In [35]:
dist_array.shape

(499500,)